In [1]:
# import uvicorn
# from fastapi import FastAPI
import json
from typing import Union, List
import re


from db_connectors import PostgresConnection, pg_pool
from schemas import *
from query_search import fields_update, search_query

# app = FastAPI()

pg = PostgresConnection(pg_pool)

In [2]:
# @app.get('/')
def home():
    return {'key':'Hello world2 '}


# @app.get('/filters')
def filters():
    query_system = """
    SELECT DISTINCT
        it_code,
        name_system,
        status,
        owner_company,
        international_name,
        identifier,
        sg_global_id,
        alternative_name,
        producer,
        domain_owner_it_system,
        it_system_owner,
        responsible_employee_system_owner,
        development_department,
        responsible_for_development,
        application_manager,
        information_security_officer,
        "date",
        development_start_date,
        commissioning_date,
        start_date_shutdown,
        decommissioning_date,
        type_of_use,
        development_type,
        hosting_type,
        access_category,
        sensitivity,
        sensitivity_assessment_date,
        pci_dss_processing,
        green_book_compliance,
        internet_access,
        fraud_sensitive,
        integirty,
        confidentiality,
        availability,
        sensivity_category,
        vital,
        rto_target,
        rpo_target,
        note,
        last_modified_date
    FROM dq_sbx.registry_it_system
    WHERE  it_code in (SELECT distinct it_code FROM dq_sbx.descr_tab)
    ORDER BY name_system
    """
    query_owner = """
    SELECT contact_person
    FROM dq_sbx.descr_col
    WHERE contact_person IS NOT NULL
    GROUP BY contact_person
    ORDER BY contact_person
    """
    query_domain = """
    SELECT domain_data_owner
    FROM dq_sbx.descr_col
    WHERE domain_data_owner IS NOT NULL
    GROUP BY domain_data_owner
    ORDER BY domain_data_owner
    """
    system = pg.getData(query_system).to_dict('list')
    owner = pg.getData(query_owner).to_dict('list')
    domain = pg.getData(query_domain).to_dict('list')
    return system, owner, domain



In [3]:
# Тестируем search_item() 

# @app.post('/query')
def search_item(item: User_Query):
    item = User_Query(**item)
    item = fields_update(item)
    query = search_query(item)
    return query #pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()



search ={
'fields_col': ["col_name", "col_comment"],
'fields_tab': ["table", "tab_comment"],
'filters_col': [],
'filters_tab': [],
'limit': 300,
'odpp': True,
'query': "dict.department"
}

# Протестируем сам запрос к базе, что бы понять что он правильный
print(search_item(search))


        

            SELECT
            it_code, src_table, tab_comment, table_code,
            array_agg(coalesce(schema_name||'.'||table_name, 'NULL')||','||src_schema||'.'||src_table) as search_val
            FROM dq_sbx.descr_tab
            WHERE  1=1 
                AND (schema_name||'.'||table_name='dict.department' OR src_schema||'.'||src_table= 'dict.department')
                
                 AND table_name IS NOT NULL 
                
            GROUP BY it_code, src_table, table_code, tab_comment

            UNION
            (
            SELECT
            it_code, src_table, tab_comment, table_code, NULL as search_val

            FROM dq_sbx.descr_tab
            WHERE  1=1 
                 and 
                ( tab_comment like '%dict.department%' 
                OR table_name like '%dict.department%' 
                OR src_table like '%dict.department%' 
                 ) 
                
                 AND table_name IS NOT NULL 
                
 

In [4]:
# df из запроса
pg.getData(search_item(search))

it_code                            src_table  \
0   ci384677                    portf_hr_employee   
1   ci384677               hr_employee_assignment   
2   ci466068                        department_pk   
3   ci384677                    hr_employee_portf   
4   ci466068                          vw_employee   
5   ci139032                    premium_employees   
6   ci384677               hr_employee_assignment   
7   ci384677                      rule_department   
8   ci384677                          hr_employee   
9   ci466068  pro_prodb_bwc_dbo_premium_employees   
10  ci384677                          hr_employee   
11  ci466068                vw_portfolio_employee   
12  ci384677                        hr_department   
13  ci466068                 rdwh_dwh_hr_employee   
14  ci466068      rdwh_dwh_hr_employee_assignment   
15  ci466068                  department_property   
16  ci384677                        hr_department   
17  ci466068               rdwh_dwh_hr_department   

                                          tab_comment  \
0                                                None   
1                                                None   
2                                                None   
3                                                None   
4                                                None   
5                                                None   
6                                                None   
7                                                None   
8                                                None   
9                                                None   
10                                               None   
11                                               None   
12                                               None   
13                                               None   
14                                               None   
15  расширенные исторические данные по справочнику...   
16                                               None   
17                                               None   

                                           table_code  \
0                  ci384677_rdwh_dm_portf_hr_employee   
1            ci384677_rdwh_dwh_hr_employee_assignment   
2               ci466068_greenplum_dict_department_pk   
3                 ci384677_rdwh_dwh_hr_employee_portf   
4              ci466068_greenplum_dm_temp_vw_employee   
5            ci139032_proportal_dbo_premium_employees   
6      ci384677_retail dwh_dwh_hr_employee_assignment   
7                   ci384677_rdwh_spr_rule_department   
8                       ci384677_rdwh_dwh_hr_employee   
9   ci466068_greenplum_stg_pro_prodb_bwc_dbo_premi...   
10                ci384677_retail dwh_dwh_hr_employee   
11   ci466068_greenplum_dm_temp_vw_portfolio_employee   
12              ci384677_retail dwh_dwh_hr_department   
13        ci466068_greenplum_stg_rdwh_dwh_hr_employee   
14  ci466068_greenplum_stg_rdwh_dwh_hr_employee_as...   
15         ci466068_greenplum_dds_department_property   
16                    ci384677_rdwh_dwh_hr_department   
17      ci466068_greenplum_stg_rdwh_dwh_hr_department   

                                           search_val  
0   [id_department,ссылка на таблицу dict.department]  
1   [id_department,ссылка на таблицу dict.department]  
2   [id_department,ссылка на таблицу dict.department]  
3   [id_department,ссылка на таблицу dict.department]  
4   [id_department,ссылка на таблицу dict.department]  
5   [id_department,ссылка на таблицу dict.department]  
6   [id_department,ссылка на таблицу dict.department]  
7   [id_department,ссылка на таблицу dict.department]  
8   [id_department,ссылка на таблицу dict.department]  
9   [id_department,ссылка на таблицу dict.department]  
10  [id_department,ссылка на таблицу dict.department]  
11  [id_department,ссылка на таблицу dict.department]  
12  [id_department,ссылка на таблицу dict.department]  
13  [id_department,ссылка на таблицу dict.dep

In [5]:
# Примерный вид как это вернутся в веб интерфейс

pg.getData(search_item(search)).columns.tolist(), pg.getData(search_item(search)).values.tolist()

(['it_code', 'src_table', 'tab_comment', 'table_code', 'search_val'],
 [['ci384677',
   'portf_hr_employee',
   None,
   'ci384677_rdwh_dm_portf_hr_employee',
   ['id_department,ссылка на таблицу dict.department']],
  ['ci384677',
   'hr_employee_assignment',
   None,
   'ci384677_rdwh_dwh_hr_employee_assignment',
   ['id_department,ссылка на таблицу dict.department']],
  ['ci466068',
   'department_pk',
   None,
   'ci466068_greenplum_dict_department_pk',
   ['id_department,ссылка на таблицу dict.department']],
  ['ci384677',
   'hr_employee_portf',
   None,
   'ci384677_rdwh_dwh_hr_employee_portf',
   ['id_department,ссылка на таблицу dict.department']],
  ['ci466068',
   'vw_employee',
   None,
   'ci466068_greenplum_dm_temp_vw_employee',
   ['id_department,ссылка на таблицу dict.department']],
  ['ci139032',
   'premium_employees',
   None,
   'ci139032_proportal_dbo_premium_employees',
   ['id_department,ссылка на таблицу dict.department']],
  ['ci384677',
   'hr_employee_assignme

In [6]:
# @app.post('/table')
def table_item(item: Table_Info):
    """Информация по таблицам"""
    item = Table_Info(**item)
    query = f"""
        SELECT
        d.it_code,d.src_name,d.src_schema,d.src_table,d.tab_comment,d.schema_name,
        d.table_name,d.is_in_archive,i.last_state,i.last_dt_from,i.load_mode,d.table_code,
        d.editable, d.team, d.developer_rb, d.developer_fio, d.analyst_rb, d.analyst_fio, d.link_confluence
        FROM dq_sbx.descr_tab as d
        LEFT JOIN dq_sbx.info as i ON d.schema_name = i.schema_name and d.table_name = i.table_name
        WHERE table_code = '{item.table_code}'"""
    return pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()
    
# @app.post('/column')
def column_item(item: Column_Info):
    """Информация по стобцам"""
    item = Column_Info(**item)
    if item.odpp is not None:
        query = f"""
        SELECT 
d.table_code, d.col_name, d.col_type, d.col_comment, d.flag_algorithm, d.personal_data_group, d.contact_person, d.domain_data_owner, d.user_comments, d.entity_name, p.warning
        FROM dq_sbx.descr_col as d
        LEFT JOIN dq_sbx.profiling_columns as p on d.col_name = p.t_column and schema_name = '{item.odpp.schema_name}' and table_name = '{item.odpp.table_name}'
        WHERE d.table_code = '{item.table_code}'
        ORDER BY d.col_position"""
        return pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()
    else:
        query = f"""
        SELECT 
d.table_code, d.col_name, d.col_type, d.col_comment, d.flag_algorithm, d.personal_data_group, d.contact_person, d.domain_data_owner, d.user_comments, d.entity_name, NULL as warning
        FROM dq_sbx.descr_col as d
        WHERE d.table_code = '{item.table_code}'
        ORDER BY d.col_position"""
        return pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()


In [7]:
# Протестируем column_item()


search ={
  "table_code": "ci466068_greenplum_stg_bis__sddu21",
"odpp": None
}


column_item(search)

(['table_code',
  'col_name',
  'col_type',
  'col_comment',
  'flag_algorithm',
  'personal_data_group',
  'contact_person',
  'domain_data_owner',
  'user_comments',
  'entity_name',
  'warning'],
 [['ci466068_greenplum_stg_bis__sddu21',
   'ddcode',
   'character varying(3)',
   'тип ccуды',
   'auto',
   None,
   None,
   None,
   None,
   None,
   None],
  ['ci466068_greenplum_stg_bis__sddu21',
   'ddabd',
   'character varying(4)',
   'отд. сч.ссуды',
   'auto',
   None,
   None,
   None,
   None,
   None,
   None],
  ['ci466068_greenplum_stg_bis__sddu21',
   'ddand',
   'character varying(6)',
   'ном. сч.ссуды',
   'auto',
   None,
   None,
   None,
   None,
   None,
   None],
  ['ci466068_greenplum_stg_bis__sddu21',
   'ddasd',
   'character varying(3)',
   'суф. сч.ссуды',
   'auto',
   None,
   None,
   None,
   None,
   None,
   None],
  ['ci466068_greenplum_stg_bis__sddu21',
   'ddabc',
   'character varying(4)',
   'отд. сч.погаш.',
   'auto',
   None,
   None,
   None,
 

In [8]:
# @app.post('/profiling')
def profiling_item(item: Profiling_Info):
    """Информация по профилированию"""
    item = Profiling_Info(**item)
    if item.col_name:
        query = f"""
        SELECT 
            schema_name, table_name, t_column, 
            p_distinct, p_unique, type_c, p_missing, p_zeros, p_negative, mean_c, min_c, max_c, 
            p_5, p_25, p_50, p_75, p_95, std, variance, iqr, "mode", kurtosis, skewness, histogram_y, histogram_x, 
            maximum_15, minimum_15, p_top, warning
        FROM dq_sbx.profiling_columns
        WHERE schema_name = '{item.odpp.schema_name}' AND table_name = '{item.odpp.table_name}' AND t_column = '{item.col_name}'
        """
        return query #pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()
    else:
        query = f"""
        SELECT 
            schema_name, table_name, sampling_date, 
            full_size, number_of_observations, sample_size_percent, duplicates_percent, number_of_variables, missing_cells_percent, 
            unsupported_types, numeric_types, string_types, datetime_types, boolean_types, heatmap, heatmap_columns
        FROM dq_sbx.profiling_table
        WHERE schema_name = '{item.odpp.schema_name}' AND table_name = '{item.odpp.table_name}'
        """
        return query #pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()

    

In [9]:
# Протестируем profiling_item()

q= {
  "odpp": {
    "schema_name": "hist_bis_sbl_etl",
    "table_name": "sdal_sbl_sddu"
  },
"col_name": "ddcode"
}

print(profiling_item(q))


        SELECT 
            schema_name, table_name, t_column, 
            p_distinct, p_unique, type_c, p_missing, p_zeros, p_negative, mean_c, min_c, max_c, 
            p_5, p_25, p_50, p_75, p_95, std, variance, iqr, "mode", kurtosis, skewness, histogram_y, histogram_x, 
            maximum_15, minimum_15, p_top, warning
        FROM dq_sbx.profiling_columns
        WHERE schema_name = 'hist_bis_sbl_etl' AND table_name = 'sdal_sbl_sddu' AND t_column = 'ddcode'
        


In [10]:
# @app.post('/editable_object')
def editable_object(item: List_Item):
    """Информация по редактируемым объектам"""
    item = List_Item(**item)
    item.name = re.sub(r"[^a-z_.#0-9 ]",'',item.name.lower())
    query = f"""
    SELECT 
        src_schema||'.'||src_table as object_name, object_type
    FROM dq_sbx.descr_tab
    WHERE editable = 'editable' 
        AND (src_schema||'.'||src_table ilike '{item.name}%' OR src_schema ilike '{item.name}%' OR  src_table ilike '{item.name}%')
    GROUP BY src_schema||'.'||src_table, object_type
    ORDER BY src_schema||'.'||src_table, object_type
    limit {item.limit}
        """
    return pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()
    

In [11]:
# Протестируем editable_object()

q= {
"name": "dds.f_calc",
"limit": 100
}

print(editable_object(q))

(['object_name', 'object_type'], [['dds.f_calc_agreement_deposit_property', 'function'], ['dds.f_calc_agreement_loan_property', 'function'], ['dds.f_calc_client_property_loan', 'function'], ['dds.f_calc_employee_branch', 'function'], ['dds.f_calc_employee_channel', 'function'], ['dds.f_calc_employee_role', 'function'], ['dds.f_calc_employee_role_tech', 'function'], ['dds.f_calc_plan_discount_new_employee', 'function'], ['dds.f_calc_x_agreement_carry', 'function'], ['dds.f_calc_x_employee', 'function']])


In [12]:
# @app.post('/users_rb')
def user_item(item: List_Item):
    """Информация по пользователеям"""
    item = List_Item(**item)
    item.name = re.sub(r"[^а-яёa-z0-9 ]",'',item.name.lower())
    query = f"""
    SELECT fio, rb
    FROM dq_sbx.rb_list
    WHERE fio ilike '%{item.name}%' OR rb ilike '%{item.name}%'
    ORDER BY fio
    LIMIT {item.limit}
        """
    return pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()


In [13]:
# Протестируем user_item()

# # test1
# q= {
# "name": "розумее",
# "limit": 5
# }

# test2
q= {
"name": "rb079",
"limit": 5
}


print(user_item(q))

(['fio', 'rb'], [['Абашкин Сергей Александрович', 'rb079067'], ['Абрамова Ирина Анатольевна', 'rb079606'], ['Абрамов Андрей Андреевич', 'rb079155'], ['Аверьянова Екатерина Анатольевна', 'rb079519'], ['Авраменкова Екатерина Валентиновна', 'rb079153']])


In [14]:
# @app.post('/update_editable_object')
def update_editable_object(item: Update_GP_Object):
    """Информация по редактируемым объектам"""
    item = Update_GP_Object(**item)
    item.team = re.sub(r"[^a-zA-Z_#0-9 ]",'',item.team)
    item.developer_rb = re.sub(r"[^a-zA-Z0-9_-]",'',item.developer_rb)
    item.developer_fio = re.sub(r"[^а-яёА-ЯЁa-zA-Z ]",'',item.developer_fio)
    item.analyst_rb = re.sub(r"[^a-zA-Z0-9_-]",'',item.analyst_rb)
    item.analyst_fio = re.sub(r"[^а-яёА-ЯЁa-zA-Z ]",'',item.analyst_fio)
    
    item.src_schema = re.sub(r"[^a-z0-9_-]",'',item.src_schema.lower())
    item.src_table = re.sub(r"[^a-z0-9_-]",'',item.src_table.lower())
    item.object_type = re.sub(r"[^a-z/]",'',item.object_type.lower())
    query = f"""
    UPDATE dq_sbx.descr_tab
    SET (team, developer_rb, developer_fio, analyst_rb, analyst_fio) = 
    ('{item.team}', '{item.developer_rb}', '{item.developer_fio}', '{item.analyst_rb}', '{item.analyst_fio}')
    WHERE editable = 'editable'
        AND src_schema = '{item.src_schema}' 
        AND src_table = '{item.src_table}' 
        """.replace("''", "NULL")
    print(query)
#     pg.execute(query)
    query2 = f"""
    DELETE FROM dq_sbx.gp_object WHERE object_schema='{item.src_schema}' AND object_name='{item.src_table}' AND object_type='{item.object_type}';

    INSERT INTO dq_sbx.gp_object (it_code, src_name, object_schema, object_name, object_type, team, developer_rb, analyst_rb) 
    VALUES('ci466068', 'greenplum', '{item.src_schema}', '{item.src_table}', '{item.object_type}', '{item.team}', '{item.developer_rb}', '{item.analyst_rb}');
    """.replace("''", "NULL")
    print(query2)
#     pg.execute(query2)


# Протестируем update_editable_object() только sql часть

# тест1
# q= {
#     "team": "Test_team",
#     "developer_rb": "developer_rb_test",
#     "developer_fio": "Developer Fio Test",
#     "analyst_rb": "analyst_rb_test",
#     "analyst_fio": "Analyst Fio Test",
    
#     "src_schema": "dict",
#     "src_table": "account_block_type",
#     "object_type": "table/view"
# }

# тест2
q= {
    "team": "Test_team",
    "developer_rb": "developer_rb_test",
    "developer_fio": "Developer Fio Test",
    
    "src_schema": "dict",
    "src_table": "account_block_type",
    "object_type": "table/view"
}

update_editable_object(q)


    UPDATE dq_sbx.descr_tab
    SET (team, developer_rb, developer_fio, analyst_rb, analyst_fio) = 
    ('Test_team', 'developer_rb_test', 'Developer Fio Test', NULL, NULL)
    WHERE editable = 'editable'
        AND src_schema = 'dict' 
        AND src_table = 'account_block_type' 
        

    DELETE FROM dq_sbx.gp_object WHERE object_schema='dict' AND object_name='account_block_type' AND object_type='table/view';

    INSERT INTO dq_sbx.gp_object (it_code, src_name, object_schema, object_name, object_type, team, developer_rb, analyst_rb) 
    VALUES('ci466068', 'greenplum', 'dict', 'account_block_type', 'table/view', 'Test_team', 'developer_rb_test', NULL);
    


In [15]:
# @app.post('/aba')
def aba(item: List_Item):
    """Информация по aba - возвращает aba и id_aba.Или всю строку, в зависмости от влага list_flag.
    Если list_flag - True то выпадающий список по аналогии с users_rb"""
    item = List_Item(**item)
    item.name = re.sub(r"[^a-z_0-9.#а-яё ]",'',item.name.lower())
    if item.list_flag:
        query = f"""
        SELECT id_aba, aba
        FROM cmd.aba
        WHERE aba ilike '%{item.name}%'
        ORDER BY aba
        limit {item.limit}
        """
        return pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()
    else:
        query = f"""
        SELECT *
        FROM cmd.aba
        WHERE aba ilike '%{item.name}%'
        ORDER BY aba
        limit {item.limit}
        """
        return pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()
    
# test
q= {
"name": "CMTO",
"limit": 5
}

# # test 2
# q= {
# "name": "CMTO",
# "limit": 5,
# "list_flag":False
# }


print(aba(q))

(['id_aba', 'aba'], [[1000, 'CMTO divisions management']])


In [16]:
# @app.post('/data_object_name')
def data_object_name(item: List_Item):
    """Информация по dd - возвращает data_object_name и id_dd. Или всю строку, в зависмости от влага list_flag.
    Если list_flag - True то выпадающий список по аналогии с users_rb"""  
    item = List_Item(**item)
    item.name = re.sub(r"[^a-z_0-9.#а-яё ]",'',item.name.lower())
    if item.list_flag:
        query = f"""
        SELECT id_dd, data_object_name
        FROM cmd.dd
        WHERE data_object_name ilike '%{item.name}%'
        ORDER BY data_object_name
        limit {item.limit}
        """
        return pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()
    else:
        query = f"""
        SELECT *
        FROM cmd.dd
        WHERE data_object_name ilike '%{item.name}%'
        ORDER BY data_object_name
        limit {item.limit}
        """
        return pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()


# test
q= {
"name": "amily",
"limit": 5
}

# # test 2
# q= {
# "name": "amily",
# "limit": 5,
# "list_flag":False
# }

print(data_object_name(q))

(['id_dd', 'data_object_name'], [[1001, 'Family']])


In [17]:
# @app.post('/termin')
def termin(item: List_Item):
    """Информация по BG - возвращает termin и id_bg. Или всю строку, в зависмости от влага list_flag.
    Если list_flag - True то выпадающий список по аналогии с users_rb"""
    item = List_Item(**item)
    item.name = re.sub(r"[^a-z_0-9.#а-яё ]",'',item.name.lower())
    if item.list_flag:
        query = f"""
        SELECT id_bg, termin
        FROM cmd.bg
        WHERE termin ilike '%{item.name}%'
        ORDER BY termin
        limit {item.limit}
        """
        return pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()
    else:
        query = f"""
        SELECT *
        FROM cmd.bg
        WHERE termin ilike '%{item.name}%'
        ORDER BY termin
        limit {item.limit}
        """
        return pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()


# # test
# q= {
# "name": "труд",
# "limit": 5
# }

# test
q= {
"name": "труд",
"limit": 5,
"list_flag":False
}

print(termin(q))

(['id_bg', 'termin', 'description', 'master_type', 'it_code'], [[1000, 'Доход от трудовых отношений', 'Доход полученный по трудовым договорам и иным гражданским договорам найма ФЛ', 'Переиспользуемые', None], [1003, 'Сотрудник', 'Наемный работник по трудовому договору в Организации', 'Мастер', 'ci272858']])


In [18]:
# @app.post('/name_system')
def name_system(item: List_Item):
    """Информация по it_code  - возвращает name_system и it_code"""
    item = List_Item(**item)
    item.name = re.sub(r"[^a-z_0-9.#а-яё ]",'',item.name.lower())
    query = f"""
    SELECT it_code, name_system
    FROM dq_sbx.registry_it_system
    WHERE name_system ilike '%{item.name}%' or it_code ilike '%{item.name}%'
    ORDER BY name_system
    limit {item.limit}
        """
    return pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()

# test1
q= {
"name": "бос",
"limit": 5
}

# # test 2
# q= {
# "name": "ci272",
# "limit": 5
# }

print(name_system(q))

(['it_code', 'name_system'], [['ci272858', 'босс кадровик'], ['ci430236', 'передача кадровых приказов в босс кадровик']])


In [19]:
# @app.post('/update_aba')
def update_aba(item: Update_ABA):
    """По аналогии с Update_GP_Object, но для ABA"""
    item = Update_ABA(**item)
    try:
        item.business_area = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.business_area)
        item.organisation_domain = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.organisation_domain)
        item.domain_data_owner = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.domain_data_owner)
        item.aba = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.aba)
        item.aba_description = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.aba_description)
        
        item.data_owner = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.data_owner)
        item.buisness_process = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.buisness_process)
        item.status = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.status)
        item.critical = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.critical)
        
        if item.id_aba:
            query = f"""
            UPDATE cmd.aba
            SET 
            (business_area, organisation_domain, domain_data_owner, 
            aba, aba_description, data_owner, buisness_process, 
            status, critical) =
            ('{item.business_area}', '{item.organisation_domain}', '{item.domain_data_owner}', 
            '{item.aba}', '{item.aba_description}','{item.data_owner}','{item.buisness_process}',
            '{item.status}','{item.critical}')
            WHERE id_aba = {item.id_aba}
            """.replace("''", "NULL")
            print(query)
#             pg.execute(query)
            return True
        else:
            query = f"""
            INSERT INTO cmd.aba
            ('{item.business_area}', '{item.organisation_domain}', '{item.domain_data_owner}', 
            '{item.aba}', '{item.aba_description}','{item.data_owner}','{item.buisness_process}',
            '{item.status}','{item.critical}')
            """.replace("''", "NULL")
            print(query)
#             pg.execute(query)
            return True
    except Exception as e:
        return e


# Протестируем sql часть

q= {
    "id_aba": 1014,
    "business_area": "Business Support",
    "organisation_domain": "AUDIT",
    "domain_data_owner": "Далиненко Андрей Борисович",
    "aba": "Audit reporting",
    "aba_description": "Отчетность по внутреннему аудиту",
    "data_owner": "Далиненко Андрей Борисович",
}

# q= {
#     "business_area": "Business Support",
#     "organisation_domain": "AUDIT",
#     "domain_data_owner": "Далиненко Андрей Борисович",
#     "aba": "Audit reporting",
#     "aba_description": "Отчетность по внутреннему аудиту",
#     "data_owner": "Далиненко Андрей Борисович",
# }

update_aba(q)


            UPDATE cmd.aba
            SET 
            (business_area, organisation_domain, domain_data_owner, 
            aba, aba_description, data_owner, buisness_process, 
            status, critical) =
            ('Business Support', 'AUDIT', 'Далиненко Андрей Борисович', 
            'Audit reporting', 'Отчетность по внутреннему аудиту','Далиненко Андрей Борисович',NULL,
            NULL,NULL)
            WHERE id_aba = 1014
            


True

In [20]:
# @app.post('/update_bg')
def update_bg(item: Update_BG):
    """По аналогии с update_aba, но для BG"""
    item = Update_BG(**item)
    try:
        item.termin = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.termin)
        item.description = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.description)
        item.master_type = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.master_type)
        item.it_code = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.it_code)
        
        if item.id_bg:
            query = f"""
            UPDATE cmd.bg
            SET
            (termin, description, master_type, it_code) =
            ('{item.termin}', '{item.description}', '{item.master_type}', '{item.it_code}')
            WHERE id_bg = {item.id_bg}
            """.replace("''", "NULL")
            print(query)
#             pg.execute(query)
            return True
        else:
            query = f"""
            INSERT INTO cmd.bg
            ('{item.termin}', '{item.description}', '{item.master_type}', '{item.it_code}')
            """.replace("''", "NULL")
            print(query)
#             pg.execute(query)
            return True
    except Exception as e:
        return e


# Протестируем sql часть

# q= {
#     "id_bg": 1003,
#     "termin": "Сотрудник",
#     "description": "Наемный работник по трудовому договору в Организации",
#     "master_type": "Мастер",
#     "it_code": "ci272858"
# }

q= {
    "id_bg": 1003,
    "termin": "Сотрудник",
    "description": "Наемный работник по трудовому договору в Организации"
}

update_bg(q)


            UPDATE cmd.bg
            SET
            (termin, description, master_type, it_code) =
            ('Сотрудник', 'Наемный работник по трудовому договору в Организации', NULL, NULL)
            WHERE id_bg = 1003
            


True

In [21]:
# @app.post('/update_dd')
def update_dd(item: Update_DD):
    """По аналогии с update_aba, но для DD"""
    item = Update_DD(**item)
    try:
        item.data_object_name = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.data_object_name)
        item.business_object_flag = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.business_object_flag)
        item.format_dd = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.format_dd)
        item.parent_object = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.parent_object)
        item.status_bo = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.status_bo)
        item.adoit_id = re.sub(r"[^а-яёА-ЯЁa-zA-Z0-9_.#/ ]",'',item.adoit_id)
        
        if item.id_dd:
            query = f"""
            UPDATE cmd.dd
            SET
            (data_object_name, business_object_flag, format_dd, parent_object,
            status_bo, adoit_id) =
            ('{item.data_object_name}', '{item.business_object_flag}', '{item.format_dd}', '{item.parent_object}',
            '{item.status_bo}', '{item.adoit_id}')
            WHERE id_dd = {item.id_dd}
            """.replace("''", "NULL")
            print(query)
#             pg.execute(query)
            return True
        else:
            query = f"""
            INSERT INTO cmd.dd
            ('{item.data_object_name}', '{item.business_object_flag}', '{item.format_dd}', '{item.parent_object}',
            '{item.status_bo}', '{item.adoit_id}')
            """.replace("''", "NULL")
            print(query)
#             pg.execute(query)
            return True
    except Exception as e:
        return e


# Протестируем sql часть
q= {
    "id_dd": 1008,
    "data_object_name": "Employee",
    "business_object_flag": "Yes",
    "format_dd": "BO",
    "parent_object": "person",
    "status_bo": "Кандидат",
    "adoit_id": "BIAN.BO.467"
}

# q= {
#     "data_object_name": "Employee",
#     "business_object_flag": "Yes",
#     "format_dd": "BO",
#     "status_bo": "Кандидат",
# }

update_dd(q)


            UPDATE cmd.dd
            SET
            (data_object_name, business_object_flag, format_dd, parent_object,
            status_bo, adoit_id) =
            ('Employee', 'Yes', 'BO', 'person',
            'Кандидат', 'BIAN.BO.467')
            WHERE id_dd = 1008
            


True

##### bg_aba_dd

In [22]:
# @app.post('/bg_aba_dd')
def bg_aba_dd(item: List_Item):
    """Информация по вьюхе связей bg_aba_dd_view - возвращает termin и id_bg. Или всю строку, в зависмости от влага list_flag.
    Если list_flag - True то выпадающий список по аналогии с users_rb"""
    item = List_Item(**item)
    item.name = re.sub(r"[^a-z_0-9.#а-яё ]",'',item.name.lower())
    if item.list_flag:
        query = f"""
        SELECT id_bg, termin
        FROM cmd.bg_aba_dd_view
        WHERE termin ilike '%{item.name}%'
        ORDER BY termin
        limit {item.limit}
        """
        return pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()
    else:
        query = f"""
        SELECT *
        FROM cmd.bg_aba_dd_view
        WHERE termin ilike '%{item.name}%'
        ORDER BY termin
        limit {item.limit}
        """
        return pg.getData(query).columns.tolist(), pg.getData(query).values.tolist()


# test
q= {
"name": "труд",
"limit": 5
}

# # test
# q= {
# "name": "труд",
# "limit": 5,
# "list_flag":False
# }

print(bg_aba_dd(q))

(['id_bg', 'termin'], [[1003, 'Сотрудник']])


In [25]:
# @app.post('/update_bg_aba_dd')
def update_bg_aba_dd(item: Update_BG_ABA_DD):
    """По аналогии с update_aba, но для BG"""
    item = Update_BG_ABA_DD(**item)
    try:        
        query = f"""
            UPDATE cmd.bg_aba_dd
            SET
            (id_dd, id_aba) =
            ('{item.id_dd}', '{item.id_aba}')
            WHERE id_bg = {item.id_bg}
            """.replace("'None'", "NULL")
        print(query)
#         pg.execute(query)
        return True
    except Exception as e:
        return e


# Протестируем sql часть

q= {
    "id_bg": 1003,
    "id_dd": 1008,
    "id_aba": 1065
}

# q= {
#     "id_bg": 1003,
#     "id_dd": 1008,
# }


# id_bg - ключевое поле
# q= {
#     "id_dd": 1008,
#     "id_aba": 1065
# }

update_bg(q)


            UPDATE cmd.bg_aba_dd
            SET
            (id_dd, id_aba) =
            ('1008', '1065')
            WHERE id_bg = 1003
            


True